In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the data
store = pd.read_csv('store.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv', low_memory=False)

In [2]:
# Merge the DataFrames on the 'Store' column
trainStore = pd.merge(train, store, on='Store')
trainStore.to_csv('trainStore.csv', index=False)

In [3]:
trainStore = trainStore.dropna()
trainStore.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
942,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
943,2,4,2015-07-30,5567,601,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
944,2,3,2015-07-29,6402,727,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
945,2,2,2015-07-28,5671,646,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
946,2,1,2015-07-27,6627,638,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
trainStore_encoded_label = trainStore.copy()

for column in trainStore_encoded_label.columns:
    if trainStore_encoded_label[column].dtype == 'object':
        trainStore_encoded_label[column] = label_encoder.fit_transform(trainStore_encoded_label[column])

trainStore_encoded_label.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
942,2,5,941,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
943,2,4,940,5567,601,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
944,2,3,939,6402,727,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
945,2,2,938,5671,646,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1
946,2,1,937,6627,638,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,1


In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from cuml.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import cuml

def check_cuda_availability():
    cuda_available = cuml.raft.raft_context.is_initialized()
    return cuda_available

def check_mps_availability():
    mps_pipe_dir = os.environ.get('CUDA_MPS_PIPE_DIRECTORY')
    return mps_pipe_dir is not None

# Check for CUDA availability
cuda_available = check_cuda_availability()

if cuda_available:
    print("CUDA is available. Using GPU for computation.")
else:
    print("CUDA is not available. Using CPU for computation.")

# Check for MPS availability
mps_available = check_mps_availability()

if mps_available:
    print("MPS (Multi-Process Service) is available. Using MPS for computation.")
else:
    print("MPS (Multi-Process Service) is not available.")

# Load the preprocessed data
trainStore = pd.read_csv('trainStore.csv')

# Encode categorical variables
label_encoder = LabelEncoder()
trainStore_encoded_label = trainStore.copy()

for column in trainStore_encoded_label.columns:
    if trainStore_encoded_label[column].dtype == 'object':
        trainStore_encoded_label[column] = label_encoder.fit_transform(trainStore_encoded_label[column])

# Split the dataframe into input features (X) and target variable (y)
X = trainStore_encoded_label.drop('Sales', axis=1)
y = trainStore_encoded_label['Sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_scaled)
X_test_imputed = imputer.transform(X_test_scaled)

# Define the neural network model
model = MLPRegressor(random_state=42)

# Define the hyperparameters to tune
param_grid = {
    'hidden_layer_sizes': [(64,), (128,), (64, 32), (128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'batch_size': [16, 32, 64],
    'max_iter': [100, 200, 300]
}

# Perform Grid Search to find the best parameters
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_imputed, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Use the best parameters to create an instance of the MLPRegressor model
best_model = MLPRegressor(**best_params, random_state=42)

# Train the model
best_model.fit(X_train_imputed, y_train)

# Predict on the testing set
y_pred = best_model.predict(X_test_imputed)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Best Parameters:", best_params)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)


ModuleNotFoundError: No module named 'cuml'